In [2]:
import json
import os
from openpyxl import load_workbook
from datetime import timedelta
import random
from copy import deepcopy
from django.contrib.gis.geos import LineString, Point
from django.contrib.admin.utils import flatten
import math
import datetime
import locale
from tqdm.notebook import tqdm
from PIL import Image
from io import BytesIO
from django.contrib.gis.geos import Point, LineString as DjangoLineString
import pytz

In [3]:
from simple_history.utils import bulk_create_with_history
from helpers.histories import bulk_update_with_history
from openpyxl_image_loader import SheetImageLoader

In [4]:
from helpers.route_maker import dic_to_ordered_list, unequal_point_pairs

In [5]:
def km_to_coordinates(road, km):

    road_marks = dic_to_ordered_list(road.marks)
    selected_pair = None

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] > km and pair[1]["km"] < km:
            selected_pair = pair
            break
        elif pair[1]["km"] > km and pair[0]["km"] < km:
            selected_pair = pair
            break
        elif pair[0]["km"] == km:
            return Point(pair[0]["point"]["coordinates"]), road
        elif pair[1]["km"] == km:
            return Point(pair[1]["point"]["coordinates"]), road

    # Calculate total length
    start_km = min([selected_pair[0]["km"], selected_pair[1]["km"]])
    end_km = max([selected_pair[0]["km"], selected_pair[1]["km"]])
    segment_mark_length = end_km - start_km
    distance_from_min = km - start_km
    # Check if km is decreasing
    invert_km = selected_pair[1]["km"] <= selected_pair[0]["km"]

    # Cut segment
    start_key = min((selected_pair[0]["index"], selected_pair[1]["index"]))
    end_key = max((selected_pair[0]["index"], selected_pair[1]["index"]))

    points = []
    for key in range(start_key, end_key + 1):
        points.append(
            Point(road.path[key][0], road.path[key][1], road.path[key][2])
        )

    segment = LineString(points, srid=4326)
    # Apply km inversion correction
    if invert_km:
        segment_distance = (
            1 - (distance_from_min / segment_mark_length)
        ) * segment.length
    else:
        segment_distance = (
            distance_from_min / segment_mark_length
        ) * segment.length
    # Find point
    point = segment.interpolate(segment_distance)

    return Point(point.x, point.y), road

In [6]:
def check_valid_road(road, km):

    road_marks = dic_to_ordered_list(road.marks)

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] >= km and pair[1]["km"] <= km:
            return True
        elif pair[1]["km"] >= km and pair[0]["km"] <= km:
            return True

    return False

In [7]:
def get_road_coordinates(road_name, km, direction, company):
    road_set = Road.objects.filter(
        name__contains=road_name, direction=int(direction), company=company
    )
#     print(road_set)

    # If roads in specified direction are not found, search road only
    # by name and order then by direction
    road_set_generic = Road.objects.filter(
        name__contains=road_name, company=company
    ).order_by("direction")

    # Check if KM range in road_set
#     print("Searching KM on direction...")
    valid = False
    for road in road_set:
        if check_valid_road(road, km):
#             print("Found KM on Road {}".format(road))
            valid = True
            break

    if not valid:
#         print("Searching KM without direction...")
        for road in road_set_generic:
            if check_valid_road(road, km):
#                 print("Found KM on Road {}".format(road))
                valid = True
                break

    if not valid:
        return Point(0, 0), None

    try:
        return km_to_coordinates(road, km)
    except Exception:
        return Point(0, 0), None

In [8]:
def get_connected_reference(company, resource, key, value):
    value=value.strip()
    select_options = company.custom_options[resource]['fields'][key]['select_options']['options']
    return next(a['value'] for a in select_options if a['name'].lower() == value.lower())

In [9]:
def get_value(field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['displayName']==field])
#     print(item_translation)
    if field == 'Nome da placa':
        value=value.upper()
    final_translation = {item['name']: item['value'] for item in item_translation}
#     print(final_translation)
    return final_translation[value]

In [10]:
def get_sign(field, sign_field, occ, value):
    if sign_field == 'Cor':
        value=value.capitalize()
    elif sign_field == 'Tipo de Película':
        value=value.split(' ')[1]
    values = occ.form_fields['fields']
    item_translation = flatten([item['innerFields'] for item in values if item['displayName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == sign_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

In [11]:
!cat .env

STAGE=PRODUCTION


In [12]:
# filename = 'acessos_importacao_v2'
filename='Pavimento' # 16/05/23

In [16]:
wb = load_workbook(filename + '.xlsx')
sheetname = wb.sheetnames[1]
# sheetname
ws = wb[sheetname]

In [17]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
        
    values.append(obj)

In [18]:
values[0]

{'km': 10,
 'km final': 12,
 'Status': 'Identificado',
 'Equipe/Empreiteira': 'Pavimento',
 'Encontrado em': datetime.datetime(2023, 6, 29, 0, 0),
 'Sentido': 'Norte',
 'Classe': 'Placas Pavimento de Concreto - ICP',
 'Faixa': 'Faixa 1',
 'Rodovia': 'BR-101 SP',
 'Tipo de pista': 'tipo de pista teste 1',
 'Segmento': 'segmento teste 1',
 'N° da Placa_1': 'placa 1',
 'N° da Placa_2': 'placa 2',
 'N° da Placa_3': 'placa 3',
 'N° da Placa_4': 'placa 4',
 'N° da Placa_5': 'placa 5',
 'N° da Placa_6': 'placa 6',
 'N° da Placa_7': 'placa 7',
 'N° da Placa_8': 'placa 8',
 'N° da Placa_9': 'placa 9',
 'N° da Placa_10': 'placa 10',
 'N° da Placa_11': 'placa 11',
 'Observações': 'obs teste 1',
 'Recurso_1': None,
 'Quantidade_1': None,
 'Recurso_2': None,
 'Quantidade_2': None,
 'Recurso_3': None,
 'Quantidade_3': None,
 'Recurso_4': None,
 'Quantidade_4': None,
 'Recurso_5': None,
 'Quantidade_5': None,
 'Recurso_6': None,
 'Quantidade_6': None,
 'Recurso_7': None,
 'Quantidade_7': None,
 'Recu

In [1]:
company = Company.objects.get(name='CCR - Rio Sp')

occurrence_type=OccurrenceType.objects.get(uuid='8906ec62-b9d2-4442-91fb-a2278ccb0d8d')
user=User.objects.get(username='rlcs')
utc = pytz.timezone('America/Sao_Paulo')
company,occurrence_type

DoesNotExist: Company matching query does not exist.

In [19]:
for a in tqdm(values):
    a['NPlaca'] = list(filter(None, [{'numeroplaca': a['N° da Placa_1']} if a['N° da Placa_1'] is not None else None,
                                    {'numeroplaca': a['N° da Placa_2']} if a['N° da Placa_2'] is not None else None,
                                    {'numeroplaca': a['N° da Placa_3']} if a['N° da Placa_3'] is not None else None,
                                    {'numeroplaca': a['N° da Placa_4']} if a['N° da Placa_4'] is not None else None,
                                    {'numeroplaca': a['N° da Placa_5']} if a['N° da Placa_5'] is not None else None,
                                    {'numeroplaca': a['N° da Placa_6']} if a['N° da Placa_6'] is not None else None,
                                    {'numeroplaca': a['N° da Placa_7']} if a['N° da Placa_7'] is not None else None,
                                    {'numeroplaca': a['N° da Placa_8']} if a['N° da Placa_8'] is not None else None,
                                    {'numeroplaca': a['N° da Placa_9']} if a['N° da Placa_9'] is not None else None,
                                    {'numeroplaca': a['N° da Placa_10']} if a['N° da Placa_10'] is not None else None,
                                    {'numeroplaca': a['N° da Placa_11']} if a['N° da Placa_11'] is not None else None]))

  0%|          | 0/198 [00:00<?, ?it/s]

In [23]:
values[2]

{'km': 10,
 'km final': 12,
 'Status': 'Identificado',
 'Equipe/Empreiteira': 'Pavimento',
 'Encontrado em': datetime.datetime(2023, 7, 1, 0, 0),
 'Sentido': 'Norte',
 'Classe': 'Placas Pavimento de Concreto - ICP',
 'Faixa': 'Faixa 3',
 'Rodovia': 'BR-101 SP',
 'Tipo de pista': 'tipo de pista teste 3',
 'Segmento': 'segmento teste 3',
 'N° da Placa_1': 'placa 3',
 'N° da Placa_2': 'placa 4',
 'N° da Placa_3': 'placa 5',
 'N° da Placa_4': 'placa 6',
 'N° da Placa_5': 'placa 7',
 'N° da Placa_6': 'placa 8',
 'N° da Placa_7': 'placa 9',
 'N° da Placa_8': 'placa 10',
 'N° da Placa_9': 'placa 11',
 'N° da Placa_10': 'placa 12',
 'N° da Placa_11': 'placa 13',
 'Observações': 'obs teste 3',
 'Recurso_1': None,
 'Quantidade_1': None,
 'Recurso_2': None,
 'Quantidade_2': None,
 'Recurso_3': None,
 'Quantidade_3': None,
 'Recurso_4': None,
 'Quantidade_4': None,
 'Recurso_5': None,
 'Quantidade_5': None,
 'Recurso_6': None,
 'Quantidade_6': None,
 'Recurso_7': None,
 'Quantidade_7': None,
 'Rec

In [44]:
objects=[]

for index, a in enumerate(tqdm(values)):

    point, road = get_road_coordinates(a['Rodovia'], a['km'], str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'].capitalize())) , company)
    if road.name != a['Rodovia']:
        print(road.name,a['Rodovia'],)
        
    try:
        objects.append(Reporting(
                    company=company,
                    occurrence_type=occurrence_type,
                    lane=get_connected_reference(company, 'reporting', 'lane', a['Faixa']),
                    road=road,
                    road_name=road.name,
                    direction=get_connected_reference(company, 'reporting', 'direction', a['Sentido'].capitalize()),
                    created_by=user,
                    found_at=a['Encontrado em'],
                    executed_at=a['Executado em'],
                    km=a['km'],
                    end_km_manually_specified= True if 'km final' in a else False,
                    end_km=float(a['km final']) if 'km final' in a and a['km final'] is not None else False,
                    point=point,
                    form_data={
                        'icppista': a['Tipo de pista'],
                        'segmento': a['Segmento'],
                        'invplacas': a['NPlaca'],
                        'notes': a['Observações']
                        },
                    ))
    except Exception as e:
        print(f'AQUI: km: {a["km"]} | Rodovia: {a["Rodovia"]} | Linha da Planilha: {index+2}')
        pass

  0%|          | 0/106 [00:00<?, ?it/s]

In [45]:
objects[105].__dict__

{'_state': <django.db.models.base.ModelState at 0x7f9933960d90>,
 'uuid': UUID('baf33d1d-4c6c-4a91-8a61-0c3d6c8cb1f3'),
 'number': '',
 'company_id': UUID('ec364a8e-b335-4729-8ac8-50ccabd73459'),
 'road_name': 'SP-255',
 'road_id': 1919,
 'km': 76.1,
 'end_km': None,
 'km_reference': None,
 'project_km': 0,
 'project_end_km': None,
 'point': <Point object at 0x7f99339507f0>,
 'direction': '1',
 'lane': '1',
 'track': None,
 'branch': None,
 'address': {},
 'created_by_id': UUID('508a66f3-3f58-4c4e-bc9a-fbba6078c02d'),
 'firm_id': None,
 'occurrence_type_id': UUID('7c44852b-84d8-453f-bfc9-d3e2e92602b2'),
 'form_data': {'cod_acesso': 'SP255 - 44 - S',
  'cod_novo': None,
  'lote': 29,
  'tipo_da_obra': 'Duplicação Km 48+100 ao Km 77+100',
  'localization': 76.1,
  'latitude': -21.78692,
  'longitude': -48.121463,
  'city': '3',
  'interessado': 'Grupo Molina',
  'matricula': '118224',
  'cri': '1º de Araraquara',
  'tipo_acesso': '3',
  'classificacao': '2',
  'atividade_do_acesso': '1',

In [47]:
for reporting in tqdm(objects):
    reporting.save()
    print(reporting.number)

  0%|          | 0/106 [00:00<?, ?it/s]

'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.00118
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.00119
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.00120
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.00121
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.00122
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.00123
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 1919: SP-255
VPT-IT-2

Searching KM on direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.00179
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.00180
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.00181
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.00182
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.00183
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.00184
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.00185
'NoneType' object has no attribute 'order'
Searching KM on directio

In [43]:
# for reporting, images in tqdm(objects):
#     for image in reversed(images):
#         reporting_file = ReportingFile(
#             created_by=user,
#             reporting=reporting,
#             description=image.name,
#             km=reporting.km,
#             point=reporting.point
#         )
#         reporting_file.save()
#         image_io = BytesIO(image._data())
#         im = Image.open(image_io)
#         if im.mode in ("RGBA", "P"):
#             im = im.convert("RGB")
#         thumb_io = BytesIO()
#         im.save(thumb_io, format='jpeg', quality=90)
#         reporting_file.upload.save(image.name + '.jpeg', thumb_io)

  0%|          | 0/585 [00:00<?, ?it/s]

In [ ]:
def get_connected_reference(company, resource, key, value):
    select_options = company.custom_options[resource]['fields'][key]['selectOptions']['options']
    return next(a['value'] for a in select_options if a['name'].lower() == value.lower())

In [ ]:
teste = get_connected_reference(company, 'reporting', 'lane', value)